In [23]:
import sys
sys.path.append("..")

In [24]:
import Graph as G
import Visualization as V
import DistMatrix as DM
import Skeletonize as S
import numpy as np

In [25]:
import importlib
importlib.reload(S)

<module 'Skeletonize' from '..\\Skeletonize.py'>

In [26]:
size = 10000
neighbors = 2
p = .5

In [27]:
g = G.load_knn_g(num=size, k=neighbors)
dm = DM.load_dist_matrix(size)
s_oht, lbls, forget = G.oht_labels(size, forget_percentage=p)

Loading dist matrix and labels
Loading dist matrix
Constructing knn graph


100%|██████████| 10000/10000 [00:00<00:00, 11855.39it/s]


Loading dist matrix
Encoding labels


In [28]:
importlib.reload(S)
clbls = np.copy(lbls)
clbls[forget] = -1
pg = S.from_matrix_to_pygel(g)

Converting matrix to pygel
Creating nodes


10000it [00:00, 277766.64it/s]


Connecting nodes


100%|██████████| 30886/30886 [00:00<00:00, 980519.63it/s]


In [29]:
importlib.reload(S)
ppg = S.pretty_from_matrix_to_pygel(g)

Converting matrix to pygel
Adding nodes and edges
Creating positions
Adding nodes to pygel


100%|██████████| 10000/10000 [00:00<00:00, 289826.01it/s]


Adding edges to pygel


100%|██████████| 30886/30886 [00:00<00:00, 1065094.17it/s]


In [30]:
from pygel3d import jupyter_display as jd

In [31]:
jd.display(ppg)

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(0,0,0)', 'width': 1},
 …

In [32]:
importlib.reload(S)
pretty_skel, smap = S.skeletonize_pg(ppg, dm, size)

jd.display(pretty_skel)

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(0,0,0)', 'width': 1},
 …

In [ ]:
jd.display(pretty_skel)